## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,katsuura,JP,35.1333,140.3000,75.11,85,100,11.21,light rain
1,1,mataura,NZ,-46.1927,168.8643,49.48,82,85,1.14,overcast clouds
2,2,sao felix do xingu,BR,-6.6447,-51.9950,77.86,68,44,3.53,scattered clouds
3,3,carnarvon,AU,-24.8667,113.6333,59.07,82,55,6.91,broken clouds
4,4,sioux lookout,CA,50.1001,-91.9170,84.58,36,20,6.91,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
user_choice_min_temperature=float(input('Enter minimum temperature: '))
user_choice_max_temperature=float(input('Enter maximum temperature: '))

Enter minimum temperature: 70
Enter maximum temperature: 90


In [4]:
city_data_df.dtypes


City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df['Max Temp'] >=user_choice_min_temperature) & (city_data_df['Max Temp'] < user_choice_max_temperature)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,katsuura,JP,35.1333,140.3000,75.11,85,100,11.21,light rain
2,2,sao felix do xingu,BR,-6.6447,-51.9950,77.86,68,44,3.53,scattered clouds
4,4,sioux lookout,CA,50.1001,-91.9170,84.58,36,20,6.91,few clouds
5,5,manta,EC,-0.9500,-80.7333,76.78,83,75,12.66,broken clouds
6,6,butaritari,KI,3.0707,172.7902,82.33,74,68,7.45,light rain
7,7,jamestown,US,42.0970,-79.2353,83.52,47,1,6.91,clear sky
13,13,sechura,PE,-5.5569,-80.8222,70.52,71,100,15.03,overcast clouds
14,14,seoul,KR,37.5683,126.9778,71.08,75,75,4.09,mist
17,17,itaporanga,BR,-7.3044,-38.1503,75.15,75,36,9.86,scattered clouds
18,18,san cristobal,VE,7.7669,-72.2250,71.78,99,67,2.89,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                321
City                   321
Country                318
Lat                    321
Lng                    321
Max Temp               321
Humidity               321
Cloudiness             321
Wind Speed             321
Current Description    321
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna(how="any")

In [8]:
clean_df.isnull().count()

City_ID                318
City                   318
Country                318
Lat                    318
Lng                    318
Max Temp               318
Humidity               318
Cloudiness             318
Wind Speed             318
Current Description    318
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,katsuura,JP,75.11,light rain,35.1333,140.3000,
2,sao felix do xingu,BR,77.86,scattered clouds,-6.6447,-51.9950,
4,sioux lookout,CA,84.58,few clouds,50.1001,-91.9170,
5,manta,EC,76.78,broken clouds,-0.9500,-80.7333,
6,butaritari,KI,82.33,light rain,3.0707,172.7902,
7,jamestown,US,83.52,clear sky,42.0970,-79.2353,
13,sechura,PE,70.52,overcast clouds,-5.5569,-80.8222,
14,seoul,KR,71.08,mist,37.5683,126.9778,
17,itaporanga,BR,75.15,scattered clouds,-7.3044,-38.1503,
18,san cristobal,VE,71.78,broken clouds,7.7669,-72.2250,


In [10]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,katsuura,JP,75.11,light rain,35.1333,140.3000,
2,sao felix do xingu,BR,77.86,scattered clouds,-6.6447,-51.9950,
4,sioux lookout,CA,84.58,few clouds,50.1001,-91.9170,
5,manta,EC,76.78,broken clouds,-0.9500,-80.7333,
6,butaritari,KI,82.33,light rain,3.0707,172.7902,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

# 6b. Iterate through the hotel DataFrame.
for i,row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    hotel_latitude=row['Lat']
    hotel_longitude=row['Lng']
    
    params['location']=f"{hotel_latitude},{hotel_longitude}"
    #print(params)
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url,params=params).json()
    #print(return_response)
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i,"Hotel Name"]=hotels['results'][0]['name']
    except:
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
#clean_hotel_df=hotel_df.loc[hotel_df['Hotel Name']!='null']
clean_hotel_df=hotel_df[hotel_df["Hotel Name"] != " "]
clean_hotel_df.isnull().values.any()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,katsuura,JP,75.11,light rain,35.1333,140.3000,Katsuura Hotel Mikazuki
2,sao felix do xingu,BR,77.86,scattered clouds,-6.6447,-51.9950,Rio Xingu Camping
4,sioux lookout,CA,84.58,few clouds,50.1001,-91.9170,Knobby's Fly-In Camps
5,manta,EC,76.78,broken clouds,-0.9500,-80.7333,Balandra Hotel
6,butaritari,KI,82.33,light rain,3.0707,172.7902,Isles Sunset Lodge


In [13]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 

max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))